# Module 1, Section 1: Foundational Concepts

In this section, we'll build a simple customer support system for TechHub, a fictional e-commerce electronics store. 

<div align="center">
    <img src="../../images/techhub_logo.png" width="500">
</div>

We'll start with the basics:
1. How to make LLM calls with messages
2. How to define tools that access external data (our TechHub database)
3. The manual tool calling loop (involved but educational!)

By the end, you'll understand **why** agent frameworks exist - spoiler, they automate the tedious parts we're about to implement manually

## Setup

First, let's load our environment variables (API keys).


In [1]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

## 1. Basic LLM Call

Let's start simple - just calling an LLM with a message.

**Note:** The default model is set workshop-wide via WORKSHOP_MODEL in the .env file (see config.py).


In [2]:
from langchain.chat_models import init_chat_model
from config import DEFAULT_MODEL

print(f"Using model: {DEFAULT_MODEL}")

# Initialize the model (using workshop default)
llm = init_chat_model(DEFAULT_MODEL)

# Simple string input
response = llm.invoke("What is LangChain in under 10 words?")
response.pretty_print()

Using model: anthropic:claude-haiku-4-5
================================== Ai Message ==================================

Framework for building applications with language models.


## 2. Working with Messages

LLMs work with **messages** that have roles:
- `SystemMessage`: Instructions for how the LLM should behave
- `HumanMessage`: User input
- `AIMessage`: Model responses
- `ToolMessage`: Results from tool execution (we'll see this soon!)


In [3]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Multi-turn conversation with messages
messages = [
    SystemMessage(
        content="You are a helpful customer support assistant for TechHub, an electronics store."
    ),
    HumanMessage(content="Hello!"),
    AIMessage(content="Hi! How can I help you today?"),
    HumanMessage(content="What's the status of order ORD-2024-0123?"),
]

response = llm.invoke(messages)
response.pretty_print()

================================== Ai Message ==================================

I'd be happy to help you check on that order! However, I don't have access to our order management system to look up specific order details.

Here are a few ways I can help you get that information:

1. **Check your email** - Look for an order confirmation or tracking email. It usually contains a tracking number and status updates.

2. **Visit our website** - Log into your TechHub account and go to "My Orders" to see real-time order status and tracking information.

3. **Contact our order support team** - They can access your account and provide detailed information. You can reach them by:
   - Calling our customer service line
   - Emailing support@techhub.com
   - Using the live chat feature on our website

Is there anything else I can help you with, like answering questions about our products or policies?


**Notice the problem:** The LLM can't actually look up the order! It doesn't have access to our database.

This is where **tools** come in.


## 3. Defining Tools

<div align="center">
    <img src="../../images/db_tools.png">
</div>

Tools give LLMs the ability to interact with external systems. Let's create four tools that query our TechHub database.

The `@tool` decorator from `langchain` automatically:
- Extracts the function signature for the LLM
- Uses the docstring as the tool description
- Handles the input/output formatting

In [4]:
from pathlib import Path
from langchain.tools import tool
from langchain_community.utilities import SQLDatabase

# instaniate the database connection
DB_PATH = Path("../../data/structured/techhub.db")
db = SQLDatabase.from_uri(f"sqlite:///{DB_PATH}")


# helper function
def extract_values(result):
    """Convert SQLDatabase query results (list of dicts) to list of tuples (values only)."""
    return [tuple(row.values()) for row in result]


@tool
def get_order_status(order_id: str) -> str:
    """Get status, dates, and tracking information for a specific order.

    Note: For order total, calculate from items using get_order_item_price().

    Args:
        order_id: The order ID (e.g., "ORD-2024-0123")

    Returns:
        Formatted string with order status, dates, and tracking number.
    """
    result = db._execute(
        f"""
        SELECT order_id, order_date, status, shipped_date, tracking_number
        FROM orders
        WHERE order_id = '{order_id}'
    """
    )
    result = extract_values(result)

    if not result:
        return f"Order {order_id} not found."

    order_id, order_date, status, shipped_date, tracking_number = result[0]

    response = f"Order {order_id}:\n"
    response += f"- Status: {status}\n"
    response += f"- Order Date: {order_date}\n"

    if shipped_date:
        response += f"- Shipped Date: {shipped_date}\n"
    if tracking_number:
        response += f"- Tracking Number: {tracking_number}\n"

    return response


@tool
def get_order_items(order_id: str) -> str:
    """Get list of items in a specific order with product IDs and quantities.

    Note: For pricing, use get_order_item_price() for historical price paid, or get_product_info() for current price.

    Args:
        order_id: The order ID (e.g., "ORD-2024-0123")

    Returns:
        Formatted string with product IDs and quantities (no prices).
    """
    result = db._execute(
        f"""
        SELECT product_id, quantity
        FROM order_items
        WHERE order_id = '{order_id}'
    """
    )
    result = extract_values(result)

    if not result:
        return f"No items found for order {order_id}."

    response = f"Items in order {order_id}:\n"
    for product_id, quantity in result:
        response += f"- Product ID: {product_id}, Quantity: {quantity}\n"

    return response


@tool
def get_product_info(product_identifier: str) -> str:
    """Get product details by product name or product ID.

    Args:
        product_identifier: Product name (e.g., "MacBook Air") or product ID (e.g., "TECH-LAP-001")

    Returns:
        Formatted string with product name, category, price, and stock status.
    """
    # Try exact ID match first
    result = db._execute(
        f"""
        SELECT product_id, name, category, price, in_stock
        FROM products
        WHERE product_id = '{product_identifier}'
    """
    )
    result = extract_values(result)

    # If no exact match, try fuzzy name search
    if not result:
        result = db._execute(
            f"""
            SELECT product_id, name, category, price, in_stock
            FROM products
            WHERE name LIKE '%{product_identifier}%'
            LIMIT 1
        """
        )
        result = extract_values(result)

    if not result:
        return f"Product '{product_identifier}' not found."

    product_id, name, category, price, in_stock = result[0]
    stock_status = "In Stock" if in_stock else "Out of Stock"

    return f"{name} ({product_id})\n- Category: {category}\n- Price: ${price:.2f}\n- Status: {stock_status}"


@tool
def get_order_item_price(order_id: str, product_id: str) -> str:
    """Get the historical price paid for a specific item in an order.

    Use this to get the actual price the customer paid at time of purchase,
    which may differ from the current retail price from get_product_info().

    Args:
        order_id: The order ID (e.g., "ORD-2024-0123")
        product_id: The product ID (e.g., "TECH-LAP-001")

    Returns:
        Formatted string with historical price per unit.
    """
    result = db._execute(
        f"""
        SELECT price_per_unit, quantity
        FROM order_items
        WHERE order_id = '{order_id}' AND product_id = '{product_id}'
        """
    )
    result = extract_values(result)

    if not result:
        return f"Item {product_id} not found in order {order_id}."

    price, quantity = result[0]
    return f"Historical price for {product_id} in {order_id}: ${price:.2f} per unit (quantity: {quantity})"

Let's test out the `get_order_status` tool

In [5]:
example_order = "ORD-2024-0127"
result = get_order_status.invoke(example_order)
print(result)

Order ORD-2024-0127:
- Status: Delivered
- Order Date: 2024-12-12
- Shipped Date: 2024-12-15
- Tracking Number: 1Z999AA174245343



Let's inspect how the `@tool` decorator parses the tool information into a `StructuredTool` object.


In [6]:
print("\n--- TOOL NAME ---")
print(get_order_status.name)

print("\n--- TOOL DESCRIPTION ---")
print(get_order_status.description)

print("\n--- TOOL ARGUMENTS ---")
print(get_order_status.args)


--- TOOL NAME ---
get_order_status

--- TOOL DESCRIPTION ---
Get status, dates, and tracking information for a specific order.

Note: For order total, calculate from items using get_order_item_price().

Args:
    order_id: The order ID (e.g., "ORD-2024-0123")

Returns:
    Formatted string with order status, dates, and tracking number.

--- TOOL ARGUMENTS ---
{'order_id': {'title': 'Order Id', 'type': 'string'}}


## 4. The Manual Tool Calling Loop

Now let's see how LLMs actually [use tools](https://docs.langchain.com/oss/python/langchain/models#tool-calling)! This happens in stages:

1. **Give the LLM access to tools** - Bind tools so the LLM knows what's available
2. **LLM decides when to call a tool** - Based on tool descriptions and the user's query
3. **LLM formats the function call** - Returns which tool to call and with what arguments (but doesn't execute it!)
4. **We manually execute the tool** - Run the actual function to get results
5. **Pass results back to the LLM** - So it can use them to generate a final answer

Let's see each stage in action!


### Step 1: Bind Tools to the Model

First, we tell the LLM what tools are available by "binding" them:


In [10]:
# Bind tools to the model - this tells the LLM what tools are available
tools = [get_order_status, get_order_items, get_product_info, get_order_item_price]
llm_with_tools = llm.bind_tools(tools)

### Step 2: LLM Decides to Call a Tool

Now let's give the LLM a query. It will decide which tool to call based on the descriptions:


In [12]:
# Ask about an order
query = "What's the status of order ORD-2024-0123?"

messages = [
    SystemMessage(content="You are a helpful customer support assistant."),
    HumanMessage(content=query),
]

# Call the LLM
response = llm_with_tools.invoke(messages)

# Add the AI's tool call to messages
messages.append(response)

response.pretty_print()

================================== Ai Message ==================================

[{'id': 'toolu_01EN7mt82DuSqmR2UaQci2YB', 'input': {'order_id': 'ORD-2024-0123'}, 'name': 'get_order_status', 'type': 'tool_use'}]
Tool Calls:
  get_order_status (toolu_01EN7mt82DuSqmR2UaQci2YB)
 Call ID: toolu_01EN7mt82DuSqmR2UaQci2YB
  Args:
    order_id: ORD-2024-0123


### Step 3: Manually Execute the Tool

The LLM told us WHAT to call and WITH WHAT ARGUMENTS. Now **we** need to actually run the function:


In [16]:
# Extract the tool call information
tool_call = response.tool_calls[0]
tool_name = tool_call["name"]
tool_args = tool_call["args"]

print(f"[Executing]\n {tool_name}(**{tool_args})", "\n")

# Manually execute the tool
if tool_name == "get_order_status":
    tool_result = get_order_status.invoke(tool_args)
elif tool_name == "get_order_items":
    tool_result = get_order_items.invoke(tool_args)
elif tool_name == "get_product_info":
    tool_result = get_product_info.invoke(tool_args)
elif tool_name == "get_order_item_price":
    tool_result = get_order_item_price.invoke(tool_args)

print(f"[Tool Result]\n {tool_result}")

[Executing]
 get_order_status(**{'order_id': 'ORD-2024-0123'}) 

[Tool Result]
 Order ORD-2024-0123:
- Status: Delivered
- Order Date: 2024-12-05
- Shipped Date: 2024-12-07
- Tracking Number: 1Z999AA113527782



### Step 4: Pass Results Back to the LLM

Now we need to send the tool's result back to the LLM so it can generate a final answer for the user. Note that we use a `ToolMessage` type when adding tool result details back to the message history.


In [17]:
from langchain_core.messages import ToolMessage

# Create a ToolMessage with the result
tool_message = ToolMessage(
    content=str(tool_result),
    tool_call_id=tool_call["id"],  # Must match the ID from the tool call
)
messages.append(tool_message)
messages

[SystemMessage(content='You are a helpful customer support assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="What's the status of order ORD-2024-0123?", additional_kwargs={}, response_metadata={}),
 AIMessage(content=[{'id': 'toolu_01EN7mt82DuSqmR2UaQci2YB', 'input': {'order_id': 'ORD-2024-0123'}, 'name': 'get_order_status', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_0196T3XByrPbze9jjdwYJUAz', 'model': 'claude-haiku-4-5-20251001', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1149, 'output_tokens': 66, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-haiku-4-5-20251001', 'model_provider': 'anthropic'}, id='lc_run--1ac99215-b22c-4211-b2c0-ad619b29ab50-0', tool_calls=[{'name': 'get_order_status', 'args': {'order_id': 'ORD-202

### Step 5: Get the Final Answer

Now the LLM has the tool result and can give a complete answer in natural language to the user:


In [18]:
# Call the LLM again with the tool result
final_response = llm_with_tools.invoke(messages)

final_response.pretty_print()

================================== Ai Message ==================================

Your order **ORD-2024-0123** has been **delivered**! Here are the details:

- **Order Date:** December 5, 2024
- **Shipped Date:** December 7, 2024
- **Tracking Number:** 1Z999AA113527782

Your package has arrived at its destination. Is there anything else you'd like to know about this order?


### Putting It All Together: The Loop

In real scenarios, the LLM might need to call multiple tools or loop several times. Let's create a function that automates this process:


In [26]:
def run_agent_loop(user_query: str):
    """Run the complete tool calling loop."""

    messages = [
        SystemMessage(
            content="You are a helpful customer support assistant for TechHub."
        ),
        HumanMessage(content=user_query),
    ]

    print(f"\n{'='*60}")
    print(f"User: {user_query}")
    print(f"{'='*60}\n")

    # Keep looping until we get a final answer
    iteration = 0
    while True:
        iteration += 1

        # Call the LLM
        response = llm_with_tools.invoke(messages)

        # Check if LLM wants to use tools
        if not response.tool_calls:
            # No tools needed - we have the final answer!
            print(f"[Final Answer]\n {response.content}\n")
            break

        # LLM wants to use tools
        print(
            f"[Iteration {iteration}] LLM is calling {len(response.tool_calls)} tool(s)..."
        )
        messages.append(response)

        # Execute each tool
        for tool_call in response.tool_calls:
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]

            # Execute the tool
            if tool_name == "get_order_status":
                result = get_order_status.invoke(tool_args)
            elif tool_name == "get_order_items":
                result = get_order_items.invoke(tool_args)
            elif tool_name == "get_product_info":
                result = get_product_info.invoke(tool_args)
            elif tool_name == "get_order_item_price":
                result = get_order_item_price.invoke(tool_args)

            arg_str = ", ".join(f"{k}={v!r}" for k, v in tool_args.items())
            print(f"  → {tool_name}({arg_str})")

            # Add tool result to messages
            messages.append(
                ToolMessage(content=str(result), tool_call_id=tool_call["id"])
            )

        print()  # Blank line before next iteration

### Try It Out!

Now let's test our agent loop with different queries:


**Example 1: Simple order lookup**


In [27]:
run_agent_loop("What's the status of order ORD-2024-0123?")


User: What's the status of order ORD-2024-0123?

[Iteration 1] LLM is calling 1 tool(s)...
  → get_order_status(order_id='ORD-2024-0123')

[Final Answer]
 Your order **ORD-2024-0123** has been **delivered**! Here are the details:

- **Status:** Delivered
- **Order Date:** December 5, 2024
- **Shipped Date:** December 7, 2024
- **Tracking Number:** 1Z999AA113527782

Your order arrived successfully. If you have any questions about your delivery or the items in your order, feel free to ask!



**Example 2: Product price lookup**


In [28]:
run_agent_loop("How much does the MacBook Air cost?")


User: How much does the MacBook Air cost?

[Iteration 1] LLM is calling 1 tool(s)...
  → get_product_info(product_identifier='MacBook Air')

[Final Answer]
 The **MacBook Air M2 (13-inch, 256GB)** costs **$1,199.00** and is currently in stock at TechHub.

Is there anything else you'd like to know about this product or any other items?



**Example 3: Multiple tools in one query**


In [29]:
run_agent_loop(
    "What's the status of order ORD-2024-0123, what was in it, and how much did it cost?"
)


User: What's the status of order ORD-2024-0123, what was in it, and how much did it cost?

[Iteration 1] LLM is calling 2 tool(s)...
  → get_order_status(order_id='ORD-2024-0123')
  → get_order_items(order_id='ORD-2024-0123')

[Iteration 2] LLM is calling 1 tool(s)...
  → get_order_item_price(order_id='ORD-2024-0123', product_id='TECH-AUD-020')

[Final Answer]
 Here's the complete information for order ORD-2024-0123:

**Order Status:**
- Status: Delivered
- Order Date: December 5, 2024
- Shipped Date: December 7, 2024
- Tracking Number: 1Z999AA113527782

**What was in it:**
- 1x Product ID: TECH-AUD-020 (1 unit)

**Cost:**
- **Total: $129.00** (1 unit at $129.00)

Your order has been successfully delivered! If you need any other information about this order, feel free to ask.

